In [1]:
from akida_models.utk_face.preprocessing import load_data

# Load the dataset
x_train, y_train, x_test, y_test = load_data()

In [2]:
# For Akida inference, use uint8 raw data
x_test_akida = x_test.astype('uint8')

In [3]:
from akida_models import fetch_file
from tensorflow.keras.models import load_model

# Retrieve the model file from the BrainChip data server
model_file = fetch_file(fname="vgg_utk_face.h5",
                        origin="https://data.brainchip.com/models/AkidaV2/vgg/vgg_utk_face.h5",
                        cache_subdir='models')

# Load the native Keras pre-trained model
model_keras = load_model(model_file)
model_keras.summary()

Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv_0 (Conv2D)             (None, 30, 30, 32)        864       
                                                                 
 conv_0/BN (BatchNormalizati  (None, 30, 30, 32)       128       
 on)                                                             
                                                                 
 conv_0/relu (ReLU)          (None, 30, 30, 32)        0         
                                                                 
 conv_1 (Conv2D)             (None, 30, 30, 32)        9216      
                                                      

In [5]:
# Compile the native Keras model (required to evaluate the MAE)
model_keras.compile(optimizer='Adam', loss='mae')

# Check Keras model performance
mae_keras = model_keras.evaluate(x_test, y_test, verbose=0)

print("Keras MAE: {0:.4f}".format(mae_keras))

Keras MAE: 6.0806


In [4]:
from akida_models import vgg_utk_face_pretrained

# Load the pre-trained quantized model
model_quantized_keras = vgg_utk_face_pretrained()
model_quantized_keras.summary()

Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 32, 32, 3)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 30, 30, 32)        896       
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 30, 30, 32)       64        
                                                                 
 conv_1 (QuantizedConv2D)    (None, 30, 30, 32)        9248      
                                                                 
 conv_1/maxpool (QuantizedMa  (None, 15, 15, 32)       0         
 xPool2D)                                             

In [7]:
import quantizeml
import json
from quantizeml.models import quantize, QuantizationParams,dump_config
#model_keras = model_keras_quantized
qparams = QuantizationParams(input_weight_bits=8, weight_bits=8, activation_bits=8, output_bits=8,
                             per_tensor_activations=True, buffer_bits=24)

# Quantize the model
quantized_model = quantize(model_keras, qparams=qparams)
quantized_model.summary()

1024/1024 [==============================] - 2s 2ms/step
Model: "vgg_utk_face"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (QuantizedRescali  (None, 32, 32, 3)        0         
 ng)                                                             
                                                                 
 conv_0 (QuantizedConv2D)    (None, 30, 30, 32)        896       
                                                                 
 conv_0/relu (QuantizedReLU)  (None, 30, 30, 32)       2         
                                                                 
 conv_1 (QuantizedConv2D)    (None, 30, 30, 32)        9248      
                                                                 
 conv_1/maxpool (QuantizedMa  (None, 15, 15, 32)       0       

In [8]:
# Compile the quantized Keras model (required to evaluate the MAE)
quantized_model.compile(optimizer='Adam', loss='mae')

# Check Keras model performance
mae_quant = quantized_model.evaluate(x_test, y_test, verbose=0)

print("Keras MAE: {0:.4f}".format(mae_quant))

Keras MAE: 6.0234


In [9]:
from cnn2snn import convert

# Convert the model
model_akida = convert(quantized_model)
model_akida.summary()

                Model Summary                 
______________________________________________
Input shape  Output shape  Sequences  Layers
[32, 32, 3]  [1, 1, 1]     1          8     
______________________________________________

___________________________________________________________
Layer (type)                 Output shape  Kernel shape  

============ SW/conv_0-dequantizer_1 (Software) ===========

conv_0 (InputConv2D)         [30, 30, 32]  (3, 3, 3, 32) 
___________________________________________________________
conv_1 (Conv2D)              [15, 15, 32]  (3, 3, 32, 32)
___________________________________________________________
conv_2 (Conv2D)              [15, 15, 64]  (3, 3, 32, 64)
___________________________________________________________
conv_3 (Conv2D)              [8, 8, 64]    (3, 3, 64, 64)
___________________________________________________________
conv_4 (Conv2D)              [1, 1, 84]    (3, 3, 64, 84)
__________________________________________________________

In [10]:
import numpy as np
from timeit import default_timer as timer
start = timer()
# Check Akida model performance
y_akida = model_akida.predict(x_test_akida)
end = timer()
print(f'Prediction time is {end-start:.2f} s.\n')
# Compute and display the MAE
mae_akida = np.sum(np.abs(y_test.squeeze() - y_akida.squeeze())) / len(y_test)
print("Akida MAE: {0:.4f}".format(mae_akida))

# For non-regression purposes
assert abs(mae_keras - mae_akida) < 0.5

Prediction time is 2.70 s.

Akida MAE: 6.0234
